In [1]:
import numpy as np

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
import torch.utils.data as utils
from torchvision import datasets, transforms
from torch.utils.data.sampler import SubsetRandomSampler
import os
import platform
import pickle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [9]:
#!rm -r DLAV-2024
#!git clone https://github.com/vita-epfl/DLAV-2024.git
path = os.getcwd() + '/../../DLAV-2024/homeworks/hw2/test_batch'

In [27]:
class ConvNet(nn.Module):
    def __init__(self, n_input_channels=3, n_output=10):
        super().__init__()
        ################################################################################
        # TODO:                                                                        #
        # Define 2 or more different layers of the neural network                      #
        ################################################################################
        # Define convolutional layers
        self.conv1 = nn.Conv2d(in_channels=n_input_channels, out_channels=16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
        
        # Define max pooling layer
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Define fully connected layers
        self.fc1 = nn.Linear(32 * 8 * 8, 128)  # Assuming input image size is 32x32 after two max pooling layers
        self.fc2 = nn.Linear(128, n_output)
        
        ################################################################################
        #                              END OF YOUR CODE                                #
        ################################################################################
    
    def forward(self, x):
        ################################################################################
        # TODO:                                                                        #
        # Set up the forward pass that the input data will go through.                 #
        # A good activation function betweent the layers is a ReLu function.           #
        #                                                                              #
        # Note that the output of the last convolution layer should be flattened       #
        # before being inputted to the fully connected layer. We can flatten           #
        # Tensor `x` with `x.view`.                                                    #
        ################################################################################
        # First convolutional layer with ReLU activation and max pooling
        x = self.pool(F.relu(self.conv1(x)))
        
        # Second convolutional layer with ReLU activation and max pooling
        x = self.pool(F.relu(self.conv2(x)))
        
        # Flatten the output for fully connected layers
        x = x.view(-1, 32 * 8 * 8)
        
        # First fully connected layer with ReLU activation
        x = F.relu(self.fc1(x))
        
        # Second fully connected layer
        x = self.fc2(x)
        
        ################################################################################
        #                              END OF YOUR CODE                                #
        ################################################################################
        
        return x
    
    def predict(self, x):
        logits = self.forward(x)
        return F.softmax(logits,dim=1)

In [31]:
def predict_usingCNN(X,model_path):
    #########################################################################
    # TODO:                                                                 #
    # - Load your saved model                                               #
    # - Do the operation required to get the predictions                    #
    # - Return predictions in a numpy array                                 #
    # Note: For the predictions, you have to return the index of the max    #
    # value                                                                 #
    #########################################################################
    net = ConvNet()

    net.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

    y_pred = net.predict(X).data.max(1)[1].numpy()
    #########################################################################
    #                       END OF YOUR CODE                                #
    #########################################################################
    return y_pred
   

In [10]:
## Read DATA
def load_pickle(f):
    version = platform.python_version_tuple()
    if version[0] == '2':
        return  pickle.load(f)
    elif version[0] == '3':
        return  pickle.load(f, encoding='latin1')
    raise ValueError("invalid python version: {}".format(version))

def load_CIFAR_batch(filename):
  """ load single batch of cifar """
  with open(filename, 'rb') as f:
    datadict = load_pickle(f)
    X = datadict['data']
    Y = datadict['labels']
    X = X.reshape(10000, 3, 32, 32).astype("float")
    Y = np.array(Y)
    return X, Y
test_filename = path
X,Y = load_CIFAR_batch(test_filename)

In [37]:
# Data Manipulation
mean_pytorch = np.array([0.4914, 0.4822, 0.4465])
std_pytorch = np.array([0.2023, 0.1994, 0.2010])
X_pytorch = np.divide(np.subtract( X/255 , mean_pytorch[np.newaxis, :,np.newaxis,np.newaxis]), std_pytorch[np.newaxis, :,np.newaxis,np.newaxis])

# Run Prediction and Evaluation
model_path = "./CNN_Classifier_7.ckpt"
prediction_cnn = predict_usingCNN(torch.from_numpy(X_pytorch).float(),model_path)
acc_cnn = sum(prediction_cnn == Y)/len(X_pytorch)
print("CNN Accuracy= %f"%(acc_cnn))

RuntimeError: Error(s) in loading state_dict for ConvNet:
	Missing key(s) in state_dict: "conv1.weight", "conv1.bias", "conv2.weight", "conv2.bias". 
	Unexpected key(s) in state_dict: "conv_1.weight", "conv_1.bias", "conv_2.weight", "conv_2.bias", "conv_3.weight", "conv_3.bias", "conv_4.weight", "conv_4.bias", "conv_5.weight", "conv_5.bias", "conv_6.weight", "conv_6.bias". 